In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import HTML

def pretty(dataframe):
    display(dataframe.style.format({'funding': '{:,} €'}))

raw_data = pd.read_csv('1.C.1_small.csv', delimiter=';')
data = raw_data[raw_data.year == 2018].drop('year', 1)

unis = dict()
for uni in data.university.unique():
    unis[uni] = list()
    uni_data = data[data.university == uni].funding.sum()
    #if not uni.startswith("Univ") or uni_data < 5000000:
    #    data = data[data.university != uni]
 
table = pd.pivot_table(data, values='funding', index=['university'],
                       columns=['funder'], fill_value=0, #margins=True,
                       aggfunc=np.sum)

totals_abs = table.sum()
totals_avg = table.mean()
totals = totals_abs/totals_abs.sum()

sums = table.agg(['sum'], axis="columns")['sum']
sums_avg = table.agg(['mean'], axis="columns")['mean']

interesting = 1 * table.gt(sums_avg, axis="rows") \
            + 2 * table.div(sums, axis="rows").gt(totals) \
            + 4 * table.gt(totals_avg, axis="columns") 


#1: uniintern
#2: vergleichend, relativ
#3: systemisch

interesting = interesting.rename(index=str, columns={'sonstige': 'sonstige Fördergeber', 'nicht bekannt / nicht zuordenbar': 'nicht zuordenbare Fördergeber'})
headers = interesting.columns
for row in interesting.itertuples():
    (uni, *values) = row
    for idx, element in enumerate(values):
        funder = headers[idx]
        switcher = [
            '',
            '$funder ist zwar uniintern wichtig, aber andere Universitäten setzen stärker auf $funder. Für $funder ist die Universität nicht relevant.',
            'Die $uni ist von $funder stärker abhängig als andere Universitäten, auch wenn $funder uniintern keine hohe Bedeutung hat. Auch für $funder ist die Universität nicht relevant.',
            '$funder ist sowohl innerhalb der Universität als auch im relativen Vergleich mit anderen eine wichtige Geldquelle. Für $funder allerdings ist die $uni nicht relevant.',
            'Für $funder ist die Universität bedeutsam, aber innerhalb und im relativen Vergleich mit anderen Universitäten ist $funder nicht relevant.',
            'Obwohl die Universität bei $funder eine erfolgreiche Fördernehmerin ist, konzentrieren andere Universitäten stärker auf $funder. Dennoch ist $funder einer der großen Förderer .',
            'Die Universität ist erfolgreich bei $funder und auch für $funder ist die $uni relevant. Nur uniintern hat $funder keine große Bedeutung.',
            '$funder ist ein wichtiger Fördergeber, sowohl intern als auch im relativen Vergleich mit anderen Universitäten. Und auch für $funder ist die $uni eine wichtige Partnerin.'           
        ]
        
        text = switcher[element].replace('$uni', uni).replace('$funder', '<strong>' + funder + '</strong>')
        if text != '':
            text = text[0:1].capitalize() + text[1:]
            unis[uni].append((element, text, funder))
            
for uni, statements in unis.items():
    if len(statements) > 0:
        statements.sort(key=lambda tup: tup[0], reverse=True)
        old_importance = 0
        display(HTML('<h3>' + uni + '</h3><p style="font-family:verdana">'))
        text = ''
        for statement in statements:
            importance = statement[0]
            if importance == old_importance:
                text += 'Dies lässt sich ebenso über <strong>' + statement[2] + '</strong> sagen.'
            else:
                text += '<br/>' + statement[1]
            text += ' '
            old_importance = importance
            
        display(HTML(text + '</p>'))
        
        
table['Summe'] = table.agg(['sum'], axis="columns")

display(table.style.format('{:,} €'))
    
print('{:,} €'.format(table.loc[('Universität Wien', 'Summe')]))

funder,Bund (Ministerien),EU,FFG,FWF,Gemeinden und Gemeindeverbände (ohne Wien),Jubiläumsfonds der ÖNB,Länder (inkl. deren Stiftungen und Einrichtungen),"Private (Stiftungen, Vereine, etc.)",Unternehmen,andere internationale Organisationen,nicht bekannt / nicht zuordenbar,sonstige,"sonstige öffentlich-rechtliche Einrichtungen (Körperschaften, Stiftungen, Fonds etc.)",ÖAW,Summe
university,,,,,,,,,,,,,,,
Akademie der bildenden Künste Wien,"64,032 €",0 €,0 €,"724,207 €",898 €,0 €,"9,384 €","164,434 €","15,274 €",0 €,0 €,"197,870 €","237,590 €","187,966 €","1,601,655 €"
Medizinische Universität Graz,"727,683 €","2,149,800 €","1,477,194 €","4,400,454 €","40,833 €","739,128 €","4,585,604 €","1,018,910 €","32,440,936 €","145,251 €",0 €,"1,985,663 €","432,015 €","171,789 €","50,315,260 €"
Medizinische Universität Innsbruck,"3,692,311 €","3,169,485 €","939,430 €","8,230,126 €",0 €,"435,520 €","487,286 €","1,493,628 €","21,297,441 €",0 €,0 €,"365,223 €","1,960,654 €","89,968 €","42,161,072 €"
Medizinische Universität Wien,"1,672,802 €","8,347,843 €","1,469,351 €","18,825,924 €","80,000 €","1,375,660 €","2,015,327 €","9,083,278 €","21,467,168 €","173,461 €",0 €,"33,211,760 €","441,721 €","842,038 €","99,006,333 €"
Montanuniversität Leoben,"3,154,355 €","3,460,399 €","13,384,395 €","1,241,006 €",0 €,0 €,"5,337,531 €",0 €,"14,284,450 €",0 €,0 €,0 €,"2,911,762 €","132,103 €","43,906,001 €"
Technische Universität Graz,"1,270,608 €","11,016,518 €","19,046,692 €","6,572,227 €","300,202 €","45,000 €","1,304,890 €","1,186,053 €","24,719,792 €",962 €,0 €,"1,502,674 €","11,908,659 €",0 €,"78,874,277 €"
Technische Universität Wien,"2,190,867 €","14,732,400 €","16,317,706 €","19,662,615 €","394,486 €","145,550 €","2,886,761 €",0 €,"22,101,496 €","862,147 €",0 €,"1,262,398 €","5,817,243 €","942,206 €","87,315,875 €"
Universität Graz,"828,650 €","2,907,787 €","1,520,199 €","12,896,937 €","685,429 €","1,298,103 €","2,123,043 €","631,401 €","1,721,040 €","229,541 €",0 €,"865,365 €","1,806,703 €","476,519 €","27,990,717 €"
Universität Innsbruck,"825,619 €","8,544,357 €","4,347,970 €","16,317,637 €","184,005 €","90,232 €","5,495,539 €","2,655,604 €","4,795,888 €","138,936 €",0 €,"20,815 €","4,620,633 €","1,374,713 €","49,411,948 €"


85,079,395 €


In [5]:

        
class Analysis:
    def __init__(self, datasheet, description, about, dataframe):
        self.__sheet = datasheet
        self.__data = pd.pivot_table(dataframe, fill_value=0, aggfunc=np.sum,
                                     values=datasheet.value_column_name,
                                     index=[datasheet.university_column_name],
                                     columns=about)
        if isinstance(about, list):
            self.__about = about
        else:
            self.__about = [about]
        self.__description = description
        self.__columns_total = self.__data.agg(['sum'], axis='columns')['sum']
        self.__rows_total = self.__data.agg(['sum'], axis='rows')
        self.__universities = dataframe[datasheet.university_column_name].unique()
        
    horizontal = "rows"
    vertical = "columns"
        
    @property
    def table(self):
        return self.__data
    @property
    def description(self):
        return self.__description
    @property
    def datasheet(self):
        return self.__sheet  
    @property
    def about(self):
        return self.__about
    @property
    def horizontal_total(self):
        return self.__columns_total
    @property
    def vertical_total(self):
        return self.__rows_total    
    @property
    def universities(self):
        return self.__universities
        
class Ranking:
    def __init__(self, analysis):
        self.__analysis = analysis
        self.__ranking = analysis.horizontal_total.div(analysis.horizontal_total.sum()). \
                            sort_values(ascending=False).reset_index()
        self.__ranking = self.__ranking.rename(index=int, columns={'sum': 'value'})
        self.__ranking.index += 1
    
    @property
    def ranking(self):
        return self.__ranking
    
    def rank_of(self, university):
        assert university in self.__analysis.universities, university + " is not included in this analysis"
        ranking = self.ranking
        idx = ranking.loc[ranking['index'] == university].index
        assert len(idx) == 1
        return int(idx[0])
    
    def better_rank_than(self, university):
        rank = int(self.rank_of(university))
        return self.top(rank-1)
    
    def top(self, rank):
        ranking = self.ranking
        return ranking.loc[ranking.index <= rank]        
        
        
class Structure:
    def __init__(self, analysis):
        self.__analysis = analysis
        #self.__avg = analysis.dataframe.agg(['mean'], axis='columns')['mean']
        self.__structure = analysis.table.div(analysis.horizontal_total, axis=Analysis.horizontal)
        self.__total_structure = analysis.vertical_total.div(analysis.vertical_total.iloc[0].sum())      
        self.__reverse_structure = analysis.table.div(analysis.vertical_total.iloc[0])
        self.__reverse_total_structure = analysis.horizontal_total.div(analysis.horizontal_total.sum()). \
                                            sort_values(ascending=False).reset_index()
        self.__reverse_total_structure.index += 1       
        self.__similarity_matrix = Similarity(analysis, self.__structure)
        #display(self.__structure.style.format('{0:.1%}'))
        #display(self.__reverse_structure.style.format('{0:.1%}'))
        #display(self.__total_structure.style.format('{0:.1%}'))        
        #print(self.__reverse_total_structure)
        
    @property
    def total(self):
        assert self.__analysis.horizontal_total.sum() == self.__analysis.vertical_total.iloc[0].sum()
        return self.__analysis.horizontal_total.sum()
    @property
    def table(self):
        return self.__structure
    @property
    def similarities(self):
        return self.__similarity_matrix
    
    def structure_of(self, university):
        assert university in self.__analysis.universities, university + " is not included in this analysis"
        tmp = self.__structure.loc[university].sort_values(ascending=False).reset_index()
        tmp.index += 1
        return tmp

class Datasheet:
    def __init__(self, dataframe, dimensions, timepoint_name, university_name, value_name):
        assert timepoint_name in dataframe, "There is no column '" + timepoint_name + "'"
        assert university_name in dataframe, "There is no column '" + university_name + "'"
        assert value_name in dataframe, "There is no colum with valid data called '" + value_name + "'"
        self.__dataframe = dataframe
        self.__timepoint_column = timepoint_name
        self.__university_column = university_name
        self.__value_column = value_name       
        self.__dimensions = Dimension.create(dataframe, dimensions)
        self.__timepoints = dataframe[timepoint_name].unique()
        self.__universities = dataframe[university_name].unique()
        self.__current_timepoint = dataframe[timepoint_name].max()
        
    @property
    def value_column_name(self):
        return self.__value_column
    @property
    def university_column_name(self):
        return self.__university_column
    @property
    def most_recent(self):
        return self.__current_timepoint
    
    def only(self, conditions):
        df = self.__dataframe
        for cond_column, cond_filter in conditions.items():
            if isinstance(cond_filter, list):
                df = df[df[cond_column].isin(cond_filter)]
            else:
                df = df[df[cond_column] == cond_filter]
        return df
    
    def analyze(self, description, about, filtering):
        return Analysis(self, description, about, self.only(filtering))        
              
        
            
class Similarity:
    def __init__(self, analysis, table):
        self.__table = table
        self.__analysis = analysis
        self.__matrix = self.__create_matrix(analysis)
        self.__num_categories = len(analysis.table.columns)
        
    def __create_matrix(self, analysis):
        unis = analysis.universities
        similarities = None
        for uni in analysis.universities: 
            if similarities is None:
                similarities = self.__to(uni)
            else:
                similarities = similarities.append(self.__to(uni))
        
        similarities.replace([np.inf, -np.inf], 0, inplace=True)
        similarities.set_index('base')
        return similarities
        
    @property
    def matrix(self):
        return self.__matrix
        
    def __to(self, university):
        tbl = self.__table
        base_row = tbl.loc[university]
        
        similarities = tbl.div(base_row)**(1/2)
        similarities = similarities.applymap(lambda x: 1/x if x > 1 else x).sum(Analysis.vertical)
        similarities.name = 'similarity'        
        similarity = (similarities / similarities[university]).to_frame().reset_index()
        similarity['base'] = university
        return similarity
    
    def to(self, university):
        m = self.matrix
        return m.loc[m['base'] == university] \
                .sort_values(by='similarity', ascending=False) \
                .set_index('university') \
                .drop(columns='base')[1:] 
                
    
    def most_similar_to(self, university, quantile=0.75):
        m = self.to(university)
        q = self.__matrix.similarity.quantile(quantile)
        return m.loc[m.similarity > q]
        
funding = Datasheet(raw_data, ['funder', 'fos'], 'year', 'university', 'funding')


analysis2 = funding.analyze('Forschungsprofil', 'fos', {'year': funding.most_recent})
s2 = Structure(analysis2)

#analysis = funding.analyze('Drittmittelanalyse', 'funder', {'year': funding.most_recent})
#uni = "Universität Wien"

#s = Structure(analysis)
#r = Ranking(analysis)

for u in analysis2.universities:
#    print("Die ähnlichste Drittmittelstruktur der " + u + ": ")
#    display(s.similarities.most_similar_to(u).style.format({'similarity': '{:.1%}'}))
    print("Das ähnlichste Forschungsprofil der " + u + ": ")
    display(s2.similarities.most_similar_to(u).style.format({'similarity': '{:.1%}'}))

#display(r.ranking.style.format({'value': '{:.1%}'}))
#print("Gesamt: " + '{:,} €'.format(s.total) + "\n")
        
#print(uni + " ist an " + str(r.rank_of(uni)) + ". Stelle")
#print("Besser sind: ")
#display(r.better_rank_than(uni).style.format({'value': '{:.1%}'}))
        
#print("Drittmittelstruktur der " + uni + ":")
#display(s.structure_of(uni).style.format({uni: '{:.1%}'}))
#display(s.structure_of('Universität für angewandte Kunst Wien').style.format({'Universität für angewandte Kunst Wien': '{:.1%}'}))

#print("Die Top-10 Universitäten sind: ")
#display(r.top(10).style.format({'value': '{:.1%}'}))

class University:
    def __init__(self, name):
        self.__name = name   
        self.__analyses = dict()
        
    @property
    def name(self):
        return self.__name
        
    def add_analysis(self, analysis):
        self.__analyses[analysis.description] = analysis
        

univie = University("Universität Wien")
univie.add_analysis(analysis2)










Das ähnlichste Forschungsprofil der Universität für Bodenkultur Wien: 


,similarity
university,
Technische Universität Wien,66.1%
Universität Innsbruck,59.6%
Universität Klagenfurt,59.4%
Universität Wien,51.1%
Universität Linz,51.0%
Universität für Weiterbildung Krems,47.4%


Das ähnlichste Forschungsprofil der Universität Wien: 


,similarity
university,
Universität Klagenfurt,65.8%
Universität Innsbruck,62.7%
Universität Salzburg,55.3%
Universität Graz,55.1%
Universität für Bodenkultur Wien,51.1%


Das ähnlichste Forschungsprofil der Technische Universität Graz: 


...

,similarity
university,
Universität für angewandte Kunst Wien,55.2%
Universität für Musik und darstellende Kunst Wien,54.3%


Das ähnlichste Forschungsprofil der Universität für künstlerische und industrielle Gestaltung Linz: 


,similarity
university,
Universität für Weiterbildung Krems,46.8%


Das ähnlichste Forschungsprofil der Universität Mozarteum Salzburg: 


,similarity
university,
Universität für Weiterbildung Krems,53.4%
Universität für Musik und darstellende Kunst Wien,43.6%


In [41]:
from pandas import Series 
import pandas as pd 
import numpy as np

#data = pd.read_csv('1.C.1.csv', sep=';', index_col=[0, 1, 2, 3])
#data.sort_index(inplace=True)

#idx = pd.IndexSlice
#tmp = data.loc[idx[:, 'Psychologie', :, 2018], :]

#print(tmp.index.get_level_values('university').unique())
#display(data.loc['FWF'].xs(2018, level='year'))
#display(data.xs(2018, level='year'))

class Dimension:
    @classmethod
    def create(cls, dataframe, dimensions):
        dims = list()
        for dim in dimensions:
            assert dim in dataframe.index.names, "There is no index '" + dim + "'"
            cats = dataframe.index.get_level_values(dim).unique()
            dims.append(Dimension(dim, cats))
        return dims
    
    def __init__(self, name, categories):
        self.__name = name
        self.__categories = categories   
    @property
    def name (self):
        return self.__name
    @property
    def categories(self):
        return self.__categories  
    def __str__(self):
        return self.name
        
class Datasheet:
    def __init__(self, filename, dimensions, timepoint_name, university_name, value_name):
        dataframe = pd.read_csv(filename, sep=';', index_col=list(range(len(dimensions)+2)))        
        dataframe.columns = [value_name]
        dataframe.sort_index(inplace=True)

        assert timepoint_name in dataframe.index.names, "There is no index '" + timepoint_name + "'"
        assert university_name in dataframe.index.names, "There is no index '" + university_name + "'"
        assert value_name in dataframe, "There is no colum with valid data called '" + value_name + "'"
        
        self.__dataframe = dataframe
        self.__timepoint_idx = timepoint_name
        self.__university_idx = university_name
        self.__value_column = value_name       
        self.__dimensions = Dimension.create(dataframe, dimensions + [university_name, timepoint_name])
        self.__timepoints = dataframe.index.get_level_values(timepoint_name).unique()
        self.__universities = dataframe.index.get_level_values(university_name).unique()
        self.__recent_timepoint = self.__timepoints.max()
                
    @property
    def value_column(self):
        return self.__value_column
    @property
    def university_idx(self):
        return self.__university_idx
    @property
    def timepoint_idx(self):
        return self.__timepoint_idx
    @property
    def dimensions(self):
        return self.__dimensions
    @property
    def most_recent(self):
        return self.__recent_timepoint
    @property
    def universities(self):
        return self.__universities
    
    def only(self, conditions):
        df = self.__dataframe
        if conditions is None:
            return df
        slices = list()
        for dim in self.__dimensions:
            if dim.name in conditions.keys():
                slices.append(conditions[dim.name])
            else:
                slices.append(slice(None))
        return df.loc[tuple(slices), :]
        
    def analyze(self, description, about, filtering=None):
        return Analysis(self, description, about, self.only(filtering) \
                            .sum(level = [about, self.university_idx]))


class Analysis:
    def __init__(self, sheet, description, about, table):
        self.__sheet = sheet
        self.__table = table.unstack(about, fill_value=0)
        self.__about = about
        self.__desc = description
            
        self.__sums = table.sum(axis='columns')
        self.__sums.name = 'funding'
        self.__totals = table.sum(level=[about])[sheet.value_column]
        self.__grand_total = table.sum()
        self.__grand_total.name = 'grand total'
        
        assert self.__grand_total[sheet.value_column] == self.__sums.sum()
               
    @property
    def sheet(self):
        return self.__sheet
    @property
    def table(self):
        return self.__table
    @property
    def horizontal_sums(self):
        return self.__sums
    @property
    def sums(self):
        return self.horizontal_sums
    @property
    def vertical_sums(self):
        return self.__totals
    @property
    def totals(self):
        return self.vertical_sums
    @property
    def about(self):
        return self.__about
    @property
    def description(self):
        return self.__desc
    @property
    def grand_total(self):
        return self.__grand_total
    
class Structure:
    def __init__(self, analysis):
        self.__analysis = analysis
        tmp = analysis.sums.sum(level=[analysis.sheet.university_idx])
        self.__table = analysis.table.divide(tmp, axis=0)
        
    @property
    def table(self):
        return self.__table
    @property
    def analysis(self):
        return self.__analysis
    
    def most_important_to(self, university, cutoff=0.9):
        u = self.table.loc[university].sort_values(ascending=False)
        cumsum = u.cumsum()
        if cumsum.iloc[0] > cutoff:
            return u[0:1]
        else:
            return u[u.cumsum() < cutoff]

    def of(self, university):
        return self.table.loc[university].sort_values(ascending=False)
    
import operator    
    
ds = Datasheet('1.C.1_small.csv', ['funder', 'fos'], 'year', 'university', 'funding')
analysis = ds.analyze("Drittmittelstruktur", "funder", {'year': ds.most_recent})

s = Structure(analysis)
    
def test_similarity():
    uni_a = "Medizinische Universität Graz"
    uni_b = "Medizinische Universität Graz"
    a = s.most_important_to(uni_a)
    b = s.most_important_to(uni_b)
    c = (1-(a-b).abs()/a)
    c.dropna(inplace=True)
    c = c.sum() / a.count()
    display(a)
    display(b)
    display(c)

def print_similarities():
    for uni_a in ds.universities:
        a = s.most_important_to(uni_a)
        results = dict()

        others = s.table
        
        sim_matrix = (1-(a-others).abs()/a)
        sim_matrix.replace([np.inf, -np.inf], np.nan, inplace=True)
        
        result = sim_matrix.sum(axis='columns') / a.count()
        result = result.sort_values(ascending=False)[1:]
        result.name = 'similarity'
        
        result = result[result > 0.4]
        
        print("Similarities to " + uni_a + ": ")
        display(result.to_frame().style.format('{:.1%}'))
        
def old():
        for uni_b in ds.universities:
            if uni_a == uni_b:
                continue
            b = s.of(uni_b)
            c = (1-(a-b).abs()/a)
            
            c.dropna(inplace=True)
            val = c.sum() / a.count()
            results[uni_b] = val

        sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse=True)
        print("Similarities of " + uni_a + ":")
        for (name, val) in sorted_results:
            if not np.isnan(val):
                print("\t" + name + ": " + '{:.1%}'.format(val))


print_similarities()  
    
    
    
    
    

Similarities to Medizinische Universität Graz: 


,similarity
university,
Montanuniversität Leoben,41.2%


Similarities to Technische Universität Graz: 


,similarity
university,
Universität Linz,71.5%
Technische Universität Wien,70.4%
Montanuniversität Leoben,67.6%
Universität für Bodenkultur Wien,60.2%
Universität Klagenfurt,59.6%
Universität Innsbruck,51.4%
Universität für Weiterbildung Krems,51.2%
Wirtschaftsuniversität Wien,47.5%
Universität Salzburg,46.7%


Similarities to Universität Klagenfurt: 


...

,similarity
university,
Universität für Bodenkultur Wien,62.2%
Universität Klagenfurt,62.0%
Wirtschaftsuniversität Wien,50.1%
Technische Universität Wien,42.3%
Universität für künstlerische und industrielle Gestaltung Linz,41.7%
Universität Linz,40.8%


Similarities to Montanuniversität Leoben: 


,similarity
university,
Universität Linz,71.7%
Technische Universität Graz,53.0%
Universität Klagenfurt,47.4%
Medizinische Universität Wien,45.3%
Universität Graz,41.8%
Universität für Bodenkultur Wien,41.3%


Similarities to Wirtschaftsuniversität Wien: 


,similarity
university,
Universität für Weiterbildung Krems,45.7%
Universität Klagenfurt,40.9%
